# Extração da característica de relação sinal-ruído de dados de EEG

A ideia é utilizar dados fictícios de ruído e sinal "bom"., para criarmos a relação dos dois sinais e obter como resultado um sinal de interesse "limpo".

A partir deste sinal, podemos no contexto de caracterização de foco, ainda extrair os rítmos cerebrais ou então classificar sinais com a presença ou não de foco, de forma que as amostras de sinais extraídas de um buffer sejam rotuladas com com a presença ou não de foco.

Esta atividade pode ser realizada em conjunto com um classificador comumente utilizado, como é o caso do SVM. Neste caso, uma porcetagem das amostras são utilizadas para treino e o restante para teste (p.e. 30 e 70% respectivamente).

In [94]:
import mne
import numpy as np
from scipy.signal import welch
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [95]:
# Funções para converter tempos em segundos e índices
def convert_min_to_sec(time):
    minutes, seconds = map(int, time.split(":"))
    return minutes * 60 + seconds

def convert_time_range_to_index(timerange):
    start, end = map(str, timerange.split(" - "))
    new_start = convert_min_to_sec(start)
    new_end = convert_min_to_sec(end)
    index = []
    index.append(int(new_start * 250))
    index.append(int(new_end * 250))
    return index

In [96]:
# Configuração dos canais e informações do EEG
n_channels = 8
ch_types = ['eeg'] * n_channels
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage("standard_1020")

<Info | 8 non-empty values
 bads: []
 ch_names: F3, Fz, F4, C3, Cz, C4, P3, P4
 chs: 8 EEG
 custom_ref_applied: False
 dig: 11 items (3 Cardinal, 8 EEG)
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>

In [97]:
# Carregando os dados
tF = '../dataset/s5/tF/OpenBCI-RAW-2023-11-06_19-56-20.txt'
tF_ob = np.loadtxt(tF, delimiter=',', skiprows=5, usecols=range(1, 9))

basal = '../dataset/s5/tF/OpenBCI-RAW-2023-11-06_19-56-20.txt'
basal_ob = np.loadtxt(basal, delimiter=',', skiprows=5, usecols=range(1, 9))

# Definindo os intervalos de tempo
tF_timeranges = ["6:00 - 10:17"]
basal_timerange = ["1:41 - 2:41"]

tF_index = convert_time_range_to_index(tF_timeranges[0])
basal_index = convert_time_range_to_index(basal_timerange[0])

# Cortando os dados nos intervalos especificados
data_cut_tF = tF_ob[tF_index[0]:tF_index[1], :]
data_cut_basal = basal_ob[basal_index[0]:basal_index[1], :]

# Criando objetos RawArray do MNE
X = {
    'tF': mne.io.RawArray(data_cut_tF.T, info),
    'basal': mne.io.RawArray(data_cut_basal.T, info),
}

Creating RawArray with float64 data, n_channels=8, n_times=64250
    Range : 0 ... 64249 =      0.000 ...   256.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=15000
    Range : 0 ... 14999 =      0.000 ...    59.996 secs
Ready.


In [98]:
# Aplicando filtros
for key in X:
    X[key].notch_filter(freqs=60)
    X[key].filter(l_freq=4, h_freq=100)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband e

In [99]:
# Parâmetros para a janela de análise
sr = 250            # Taxa de amostragem
jump = 5            # Tamanho da janela em segundos
size = sr * jump    # Número de pontos por janela

# Definindo as bandas de frequência
theta_band = (4, 8)
alpha_band = (8, 13)
beta_band = (13, 30)
gamma_band = (30, 100)

# Inicializando listas para características e rótulos
features = []
labels = []

# Mapeamento de índices para rótulos
band_labels = {
    0: 'theta',
    1: 'alpha',
    2: 'beta',
    3: 'gamma',
}

# Estimando o ruído de fundo usando o sinal basal
ndarray_data_basal = X['basal'].copy().get_data()
_, data_basal_frequency = welch(ndarray_data_basal, fs=sr, nperseg=512, noverlap=256)
noise_power = np.mean(data_basal_frequency, axis=1)

# Estimando o ruído de fundo
estimated_background_noise = np.mean(noise_power)
print('Ruído de fundo estimado:', estimated_background_noise)

# Processando os dados do teste final (tF)
data = X['tF']
data.set_eeg_reference(ref_channels='average', projection=False)

# Inicializando amostras e resultados
samples = {'focus': [], 'not_focus': []}
results = [0] * 4  # Supondo 4 bandas (theta, alpha, beta, gamma)

# Percorrendo o sinal em janelas de 'jump' segundos com passo de 1 segundo
for i in range(0, int(data.times[-1] - jump)):
    tmin = i
    tmax = i + jump

    cut = data.copy().crop(tmin=tmin, tmax=tmax).get_data()

    freqs, psd = welch(cut, fs=sr, nperseg=128, noverlap=64)

    X_avg = np.average(psd, axis=0)

    theta_idxs = np.where((freqs >= theta_band[0]) & (freqs <= theta_band[1]))[0]
    alpha_idxs = np.where((freqs >= alpha_band[0]) & (freqs <= alpha_band[1]))[0]
    beta_idxs = np.where((freqs >= beta_band[0]) & (freqs <= beta_band[1]))[0]
    gamma_idxs = np.where((freqs >= gamma_band[0]) & (freqs <= gamma_band[1]))[0]

    bands = [
        np.sum(X_avg[theta_idxs]),
        np.sum(X_avg[alpha_idxs]),
        np.sum(X_avg[beta_idxs]),
        np.sum(X_avg[gamma_idxs])
    ]

    dominant_band = np.argmax(bands)
    results[dominant_band] += 1

    if dominant_band == 2 or dominant_band == 3:
        samples['focus'].append(X_avg)
    else:
        samples['not_focus'].append(X_avg)

    # Adicionando o rótulo correspondente
    # labels.append(band_labels.get(dominant_band))

# total = sum(results)
# percentages = [round((count/total) * 100, 2) for count in results]
# print(data.__str__())
# total_pc[data_names[k]] = percentages

# print(total_pc)
print(samples)

Ruído de fundo estimado: 3.2651063829198472
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
{'focus': [array([4.39609664e-02, 2.65592039e-01, 8.62782689e-01, 7.75970452e-01,
       6.77808991e-01, 7.64565153e-01, 8.00812259e-01, 5.97749417e-01,
       5.12516192e-01, 3.28357358e-01, 4.14515678e-01, 3.97308792e-01,
       3.23041237e-01, 3.49943955e-01, 3.81851568e-01, 2.29199826e-01,
       2.39442021e-01, 2.41573534e-01, 2.01324637e-01, 1.77293225e-01,
       1.91483773e-01, 1.79156836e-01, 1.77784169e-01, 1.70476888e-01,
       1.37903235e-01, 1.36181578e-01, 1.46189610e-01, 1.24407907e-01,
       1.08774692e-01, 9.38392780e-02, 6.57340980e-02, 8.48119911e-02,
       1.34793247e-01, 1.14164723e-01, 9.80379755e-02, 9.97610056e-02,
       7.71778314e-02, 7.06545047e-02, 7.79072403e-02, 4.58835108e-02,
       4.11225171e-02, 3.89517633e-02, 4.75602371e-02, 4.89889074e-02,
       2.77688655e-02, 3.52493758e-02, 2.88827417e-02

In [100]:
# Convertendo o tipo dos dados filtrados de objeto RawArray para ndarray
ndarray_data_basal = X['basal'].copy().get_data()

# Transformando os dados de basal para o domínio da frequência
_, data_basal_frequency = welch(ndarray_data_basal, fs=250, nperseg=512, noverlap=256)

# Ajustando o sinal alvo
target_amplitudes_adjusted = ndarray_data_basal - estimated_background_noise

# Lista que armazena as médias de potência para cada canal
noise_power = []



for channel_data in data_basal_frequency:
    # fft_result = np.fft.fft(channel_data)

    # # densidade espectral de potência (PSD)
    # psd = np.abs(fft_result) ** 2
    
    # média da potência no intervalo de tempo sem estímulo
    base_power = np.mean(channel_data)
    noise_power.append(base_power)

# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
print(estimated_background_noise)

3.2651063829198472


In [101]:
# Corrigindo o nome da variável
target_amplitudes_adjusted = ndarray_data_basal - estimated_background_noise

# Calculando o SNR de banda estreita
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

# Calculando o SNR de banda larga
total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)


[[        nan         nan         nan ... 15.01630431 13.10954691
          nan]
 [        nan         nan         nan ... 15.13807748 13.36156309
          nan]
 [        nan         nan         nan ... 15.43866366 13.87713477
          nan]
 ...
 [        nan         nan         nan ... 15.32795146 13.75856579
          nan]
 [        nan         nan         nan ... 14.87365919 13.48101537
          nan]
 [        nan         nan         nan ... 15.28385431 13.89197355
          nan]]
(8, 15000)
[[-50.75674989 -41.21870517 -39.7713109  ...          nan          nan
  -50.75674989]
 [-50.75674989 -40.84192651 -41.05988689 ...          nan          nan
  -50.75674989]
 [-50.75674989 -39.65047319 -39.97704261 ...          nan          nan
  -50.75674989]
 ...
 [-50.75674989 -40.34828502 -38.78287707 ...          nan          nan
  -50.75674989]
 [-50.75674989 -42.15402394 -39.57944345 ...          nan          nan
  -50.75674989]
 [-50.75674989 -40.49944454 -38.7227433  ...          nan

/tmp/ipykernel_10200/66287516.py:5: RuntimeWarning: invalid value encountered in log10
  narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
/tmp/ipykernel_10200/66287516.py:11: RuntimeWarning: invalid value encountered in log10
  wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)


In [102]:
# Transformando as amostras para um formato adequado para classificação
X_data = []
y_labels = []

# Iterando sobre as amostras e extraindo características
for label, sample_set in samples.items():
    for sample in sample_set:
        # Extrair características como média e mediana
        features = [
            np.mean(sample),  # Média da característica
            np.median(sample),  # Mediana da característica
            *np.mean(narrow_band_SNR, axis=1),  # Média do SNR estreito para cada canal
            *np.mean(wide_band_SNR, axis=1)  # Média do SNR largo para cada canal
        ]
        
        # Adicionando as características para X_data
        X_data.append(features)
        
        # Adicionando o rótulo para y_labels (1 = foco, 0 = não foco)
        y_labels.append(1 if label == 'focus' else 0)

# Convertendo para array numpy
X_data = np.array(X_data)
y_labels = np.array(y_labels)

# Imputação de valores NaN com a média
imputer = SimpleImputer(strategy="mean")
X_data = imputer.fit_transform(X_data)

# Dividindo os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_data, y_labels, test_size=0.3, random_state=42)

# Treinamento do classificador SVM
clf = SVC()
clf.fit(X_train, y_train)

# Predição e avaliação
y_pred = clf.predict(X_test)
print('Acurácia:', accuracy_score(y_test, y_pred))
print('Relatório de Classificação:')
print(classification_report(y_test, y_pred))

# Seletor de características
selector = RFE(clf, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)
print('Ranking das Características:')
print(selector.ranking_)

Acurácia: 1.0
Relatório de Classificação:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        76

    accuracy                           1.00        76
   macro avg       1.00      1.00      1.00        76
weighted avg       1.00      1.00      1.00        76

Ranking das Características:
[1 1]


/home/thaynoanhit/Documentos/Reconhecimento_De_Padroes/ven/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/home/thaynoanhit/Documentos/Reconhecimento_De_Padroes/ven/lib/python3.12/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=2. There will be no feature selection and all features will be kept.
  warnings.warn(
